## Installation
### Prerequisites
- Kubernetes Cluster with Windows node
- `kubectl` & `helm` installed

### Installing Argo Workflows

Installing CLI:

In [ ]:
curl.exe -o argo.gz -LOs "https://github.com/argoproj/argo-workflows/releases/download/v3.2.6/argo-windows-amd64.gz"
7z.exe x .\argo.gz
mv .\argo-windows-amd64 .\argo.exe -Force

Installing Helm Chart:

In [ ]:
helm repo add argo https://argoproj.github.io/argo-helm
helm upgrade argo-workflows argo/argo-workflows --install --version 0.9.4 -n argo --create-namespace --set server.serviceType=LoadBalancer --set workflow.serviceAccount.name=default --set controller.containerRuntimeExecutor=emissary

Getting dashboard URL and token for auth:

In [ ]:
$ip=$(kubectl get svc argo-workflows-server -n argo --template "{{ range (index .status.loadBalancer.ingress 0) }}{{.}}{{ end }}")
Write-Host "http://$($ip):2746"
.\argo.exe auth token

### Installing Tekton

Installing Tekton Pipelines with disabled affinity assistant (not working yet with Windows Containers) as well as the Tekton dashboard:

In [ ]:
kubectl apply --filename https://storage.googleapis.com/tekton-releases/pipeline/previous/v0.30.0/release.yaml
kubectl patch cm feature-flags -n tekton-pipelines -p '{\"data\": {\"disable-affinity-assistant\": \"true\"}}'

kubectl apply --filename https://storage.googleapis.com/tekton-releases/dashboard/latest/tekton-dashboard-release.yaml
# Make dasboard available via loadbalancer
kubectl patch svc tekton-dashboard -n tekton-pipelines -p '{\"spec\": {\"type\": \"LoadBalancer\"}}'

Getting dashboard URL:

In [ ]:
$ip=$(kubectl get svc tekton-dashboard -n tekton-pipelines --template "{{ range (index .status.loadBalancer.ingress 0) }}{{.}}{{ end }}")
Write-Host "http://$($ip):9097"

## Running pipelines

We build, test & create a Windows container image for our [example .NET app](./example-app/Program.cs).

For pushing the image, we add our Docker registry credentials to the cluster (assuming you don't use an external credential store):

In [ ]:
kubectl create secret generic docker-config --from-file=$env:USERPROFILE\.docker\config.json

### Cross-compiling in Linux build stage

BuildKit makes it possible to run a Linux build stage before our final Windows build stage as shown in [`example-app/Dockerfile.cross`](./example-app/Dockerfile.cross).

The pipelines  basically just need to build our container image as building the binary and running tests is included.

In [ ]:
kubectl apply -f .\pipelines\cross\argo-buildkit.yml
kubectl apply -f .\pipelines\cross\tekton-buildkit.yml

While they run you can look at the definitions:
- [`pipelines/cross/argo-buildkit.yml`](./pipelines/cross/argo-buildkit.yml)
- [`pipelines/cross/tekton-buildkit.yml`](./pipelines/cross/argo-buildkit.yml)

You can also look at the pipelines via the dashboards. As soon as they're successful, you can try out the created images with:

In [ ]:
docker run --rm --pull always lippertmarkus/test:cross-argo-buildkit
docker run --rm --pull always lippertmarkus/test:cross-tekton-buildkit

### Compile out of image build